## Calculating Symbolic 2D expressions
$$ U = cos(\theta s /2) I + i sin(\theta s /2) R $$
$$R_{xy} = cos(\phi) X + sin(\phi) Y$$


In [1]:
import sympy as sp
import numpy as np
from numpy import exp, sqrt, array
from numpy.random import multivariate_normal

In [2]:
# === Symbols ===
theta, s, phi, psi = sp.symbols('theta s phi psi', real=True)
alpha = s * theta / 2

# === Pauli Matrices and Identity ===
I2 = sp.eye(2)
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])

R = (
    sp.cos(phi)*X + sp.sin(phi)*Y
)

U = sp.cos(alpha) * I2 - sp.I * sp.sin(alpha) * R
U_dag = U.H

---
## Symbolic Calculations ##

Find components in X Y Z

In [3]:
def pauli_decomposition(Z_s ,paulis):
    components = {}
    for label, sigma in paulis.items():
        coeff = (Z_s * sigma).trace() / 2
        components[label] = sp.simplify(coeff)
    return components

paulis = {'I': I2, 'X': X, 'Y': Y, 'Z': Z}


In [4]:
def fully_simplify_pauli_decomposition(decomp, collect_vars=None):
    """
    Deeply simplify and factor a symbolic Pauli decomposition dictionary.

    Parameters:
        decomp (dict): output from pauli_decomposition
        collect_vars (list): list of sympy symbols to factor/collect

    Returns:
        dict: simplified and factored expressions
    """
    simplified = {}
    for pauli, expr in decomp.items():
        # Step-by-step simplification
        expr = sp.expand_trig(expr)
        expr = sp.trigsimp(expr)
        expr = sp.expand(expr)
        expr = sp.powsimp(expr, force=True)
        expr = sp.simplify(expr)
        expr = sp.expand_complex(expr)

        # Collect φ, ψ, or any variables provided
        if collect_vars:
            for var in collect_vars:
                expr = sp.collect(expr, var)
                if isinstance(var, sp.Basic):
                    expr = sp.collect(expr, sp.exp(sp.I * var))
                    expr = sp.collect(expr, sp.exp(-sp.I * var))
        
        simplified[pauli] = expr
    return simplified


## Z(t) Component ##

In [5]:
Z_s = U_dag * Z * U
Z_s = sp.simplify(Z_s)
Z_s = sp.expand_trig(Z_s)
Z_s = sp.trigsimp(Z_s)
Z_s

Matrix([
[                                            cos(s*theta), 2*I*(I*sin(phi) - cos(phi))*sin(s*theta/2)*cos(s*theta/2)],
[2*(-sin(phi) + I*cos(phi))*sin(s*theta/2)*cos(s*theta/2),                                             -cos(s*theta)]])

In [6]:
Z_s_decomp = pauli_decomposition(Z_s, paulis)
Z_s_decomp = sp.simplify(Z_s_decomp)
Z_s_decomp = sp.trigsimp(Z_s_decomp)
Z_s_decomp

{I: 0, X: -sin(phi)*sin(s*theta), Y: sin(s*theta)*cos(phi), Z: cos(s*theta)}

In [7]:
Z_s_decomp = pauli_decomposition(Z_s, paulis)

# Clean and factor using φ and ψ
Z_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    Z_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
Z_s_decomp_cleaned

{'I': 0,
 'X': -sin(phi)*sin(s*theta),
 'Y': sin(s*theta)*cos(phi),
 'Z': cos(s*theta)}

Finding X component

In [8]:
Z_s_decomp_cleaned['X']


-sin(phi)*sin(s*theta)

Finding Y component

In [9]:
Z_s_decomp_cleaned['Y']

sin(s*theta)*cos(phi)

Finding Y component

In [10]:
Z_s_decomp_cleaned['Z']

cos(s*theta)

## Y(t) Component ##

In [11]:
Y_s = U_dag * Y * U
Y_s = sp.simplify(Y_s)
Y_s = sp.expand_trig(Y_s)
Y_s = sp.trigsimp(Y_s)
Y_s

Matrix([
[                                 -sin(s*theta)*cos(phi), I*(-cos(s*theta/2)**2 + exp(-2*I*phi)*sin(s*theta/2)**2)],
[I*(-exp(2*I*phi)*sin(s*theta/2)**2 + cos(s*theta/2)**2),                                    sin(s*theta)*cos(phi)]])

In [12]:
Y_s_decomp = pauli_decomposition(Y_s, paulis)
Y_s_decomp = sp.simplify(Y_s_decomp)
Y_s_decomp = sp.trigsimp(Y_s_decomp)
Y_s_decomp

{I: 0, X: sin(2*phi)*sin(s*theta/2)**2, Y: -exp(2*I*phi)*sin(s*theta/2)**2/2 + cos(s*theta/2)**2 - exp(-2*I*phi)*sin(s*theta/2)**2/2, Z: -sin(s*theta)*cos(phi)}

In [13]:
Y_s_decomp = pauli_decomposition(Y_s, paulis)

# Clean and factor using φ and ψ
Y_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    Y_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
Y_s_decomp_cleaned

{'I': 0,
 'X': 2*sin(phi)*sin(s*theta/2)**2*cos(phi),
 'Y': -sin(s*theta/2)**2*cos(2*phi) + cos(s*theta/2)**2,
 'Z': -sin(s*theta)*cos(phi)}

In [14]:
Y_s_decomp_cleaned['X']


2*sin(phi)*sin(s*theta/2)**2*cos(phi)

In [15]:
Y_s_decomp_cleaned['Y']

-sin(s*theta/2)**2*cos(2*phi) + cos(s*theta/2)**2

In [16]:
Y_s_decomp_cleaned['Z']

-sin(s*theta)*cos(phi)

## X(t) Component ##

In [17]:
X_s  = U_dag * X * U
X_s  = sp.simplify(X_s )
X_s  = sp.expand_trig(X_s )
X_s  = sp.trigsimp(X_s )
X_s 

Matrix([
[                             sin(phi)*sin(s*theta), cos(s*theta/2)**2 + exp(-2*I*phi)*sin(s*theta/2)**2],
[exp(2*I*phi)*sin(s*theta/2)**2 + cos(s*theta/2)**2,                              -sin(phi)*sin(s*theta)]])

In [18]:
X_s_decomp = pauli_decomposition(X_s , paulis)
X_s_decomp = sp.simplify(X_s_decomp)
X_s_decomp = sp.trigsimp(X_s_decomp)
X_s_decomp

{I: 0, X: exp(2*I*phi)*sin(s*theta/2)**2/2 + cos(s*theta/2)**2 + exp(-2*I*phi)*sin(s*theta/2)**2/2, Y: sin(2*phi)*sin(s*theta/2)**2, Z: sin(phi)*sin(s*theta)}

In [19]:
X_s_decomp = pauli_decomposition(X_s , paulis)

# Clean and factor using φ and ψ
X_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    X_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
X_s_decomp_cleaned

{'I': 0,
 'X': sin(s*theta/2)**2*cos(2*phi) + cos(s*theta/2)**2,
 'Y': 2*sin(phi)*sin(s*theta/2)**2*cos(phi),
 'Z': sin(phi)*sin(s*theta)}

In [20]:
X_s_decomp_cleaned['X']

sin(s*theta/2)**2*cos(2*phi) + cos(s*theta/2)**2

In [21]:
X_s_decomp_cleaned['Y']

2*sin(phi)*sin(s*theta/2)**2*cos(phi)

In [22]:
X_s_decomp_cleaned['Z']

sin(phi)*sin(s*theta)

## Checking I components

In [23]:
I_s = U_dag * I2 * U
I_s = sp.simplify(I_s)
I_s = sp.expand_trig(I_s)
I_s = sp.trigsimp(I_s)
I_s

Matrix([
[1, 0],
[0, 1]])

In [24]:
I_s_decomp = pauli_decomposition(I_s, paulis)

# Clean and factor using φ and ψ
I_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    I_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
I_s_decomp_cleaned

{'I': 1, 'X': 0, 'Y': 0, 'Z': 0}

# Relaxation Contributions

## $ \sigma_- $ Channel
$$ \sigma^- = (X-iY)/2 $$

In [25]:
sigma_minus = (X - sp.I * Y) / 2
sigma_minus

Matrix([
[0, 0],
[1, 0]])

In [26]:

sigma_minus_s  = U_dag * sigma_minus * U
sigma_minus_s  = sp.simplify(sigma_minus_s )
sigma_minus_s  = sp.expand_trig(sigma_minus_s )
sigma_minus_s  = sp.trigsimp(sigma_minus_s )
sigma_minus_s 

Matrix([
[(sin(phi) + I*cos(phi))*sin(s*theta/2)*cos(s*theta/2),                         exp(-2*I*phi)*sin(s*theta/2)**2],
[                                    cos(s*theta/2)**2, I*(I*sin(phi) - cos(phi))*sin(s*theta/2)*cos(s*theta/2)]])

In [27]:
sigma_minus_s_decomp = pauli_decomposition(sigma_minus_s , paulis)
sigma_minus_s_decomp = sp.simplify(sigma_minus_s_decomp)
sigma_minus_s_decomp = sp.trigsimp(sigma_minus_s_decomp)
sigma_minus_s_decomp

{I: 0, X: cos(s*theta/2)**2/2 + exp(-2*I*phi)*sin(s*theta/2)**2/2, Y: I*(-cos(s*theta/2)**2 + exp(-2*I*phi)*sin(s*theta/2)**2)/2, Z: (sin(phi) + I*cos(phi))*sin(s*theta/2)*cos(s*theta/2)}

In [28]:
sigma_minus_s_decomp = pauli_decomposition(sigma_minus_s , paulis)

# Clean and factor using φ and ψ
sigma_minus_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    sigma_minus_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
sigma_minus_s_decomp_cleaned

{'I': 0,
 'X': -I*sin(2*phi)*sin(s*theta/2)**2/2 + sin(s*theta/2)**2*cos(2*phi)/2 + cos(s*theta/2)**2/2,
 'Y': I*(sin(s*theta/2)**2*cos(2*phi)/2 - cos(s*theta/2)**2/2) + sin(2*phi)*sin(s*theta/2)**2/2,
 'Z': sin(phi)*sin(s*theta/2)*cos(s*theta/2) + I*sin(s*theta/2)*cos(phi)*cos(s*theta/2)}

In [29]:
sigma_minus_s_decomp_cleaned['X']

-I*sin(2*phi)*sin(s*theta/2)**2/2 + sin(s*theta/2)**2*cos(2*phi)/2 + cos(s*theta/2)**2/2

In [30]:
sigma_minus_s_decomp_cleaned['Y']

I*(sin(s*theta/2)**2*cos(2*phi)/2 - cos(s*theta/2)**2/2) + sin(2*phi)*sin(s*theta/2)**2/2

In [31]:
sigma_minus_s_decomp_cleaned['Z']

sin(phi)*sin(s*theta/2)*cos(s*theta/2) + I*sin(s*theta/2)*cos(phi)*cos(s*theta/2)

## $ \sigma_+ $ Channel
$$ \sigma^+ = (X+iY)/2 $$

In [32]:
sigma_plus = (X + sp.I * Y) / 2
sigma_plus

Matrix([
[0, 1],
[0, 0]])

In [33]:

sigma_plus_s  = U_dag * sigma_plus * U
sigma_plus_s  = sp.simplify(sigma_plus_s )
sigma_plus_s  = sp.expand_trig(sigma_plus_s )
sigma_plus_s  = sp.trigsimp(sigma_plus_s )
sigma_plus_s 

Matrix([
[(sin(phi) - I*cos(phi))*sin(s*theta/2)*cos(s*theta/2),                                      cos(s*theta/2)**2],
[                       exp(2*I*phi)*sin(s*theta/2)**2, (-sin(phi) + I*cos(phi))*sin(s*theta/2)*cos(s*theta/2)]])

In [34]:
sigma_plus_s_decomp = pauli_decomposition(sigma_plus_s , paulis)
sigma_plus_s_decomp = sp.simplify(sigma_plus_s_decomp)
sigma_plus_s_decomp = sp.trigsimp(sigma_plus_s_decomp)
sigma_plus_s_decomp

{I: 0, X: (-exp(2*I*phi)*cos(s*theta) + exp(2*I*phi) + cos(s*theta) + 1)/4, Y: I*(-exp(2*I*phi)*sin(s*theta/2)**2 + cos(s*theta/2)**2)/2, Z: (sin(phi) - I*cos(phi))*sin(s*theta/2)*cos(s*theta/2)}

In [35]:
sigma_plus_s_decomp = pauli_decomposition(sigma_plus_s , paulis)

# Clean and factor using φ and ψ
sigma_plus_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    sigma_plus_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
sigma_plus_s_decomp_cleaned

{'I': 0,
 'X': I*(-sin(2*phi)*cos(s*theta)/4 + sin(2*phi)/4) - cos(2*phi)*cos(s*theta)/4 + cos(2*phi)/4 + cos(s*theta)/4 + 1/4,
 'Y': I*(-sin(s*theta/2)**2*cos(2*phi)/2 + cos(s*theta/2)**2/2) + sin(2*phi)*sin(s*theta/2)**2/2,
 'Z': sin(phi)*sin(s*theta/2)*cos(s*theta/2) - I*sin(s*theta/2)*cos(phi)*cos(s*theta/2)}

In [36]:
sigma_plus_s_decomp_cleaned['X']

I*(-sin(2*phi)*cos(s*theta)/4 + sin(2*phi)/4) - cos(2*phi)*cos(s*theta)/4 + cos(2*phi)/4 + cos(s*theta)/4 + 1/4

In [37]:
sigma_plus_s_decomp_cleaned['Y']

I*(-sin(s*theta/2)**2*cos(2*phi)/2 + cos(s*theta/2)**2/2) + sin(2*phi)*sin(s*theta/2)**2/2

In [38]:
sigma_plus_s_decomp_cleaned['Z']

sin(phi)*sin(s*theta/2)*cos(s*theta/2) - I*sin(s*theta/2)*cos(phi)*cos(s*theta/2)



---

### Compute the variance over time

We define the variance of this process as:

$$
\mathrm{Var} = \int_0^1 f(s)^2 \, ds
$$

This integral gives us a symbolic expression that depends on θ, φ, and ψ. It represents the **Itô variance** associated with the stochastic evolution.

---

### Sample from the variance

Once we evaluate the variance for specific values of θ, φ, and ψ, we obtain a real scalar σ². We can sample from the corresponding zero-mean Gaussian distribution:

$$
x \sim \mathcal{N}(0, \mathrm{Var})
$$

This sampled value represents one instance of the quantum noise coefficient aligned with this Pauli component.

---

### Construct the Hermitian noise matrix

Once we sample all 3 Pauli components (X, Y, Z), we form the Hermitian matrix:

$$
I_d = e_d \begin{bmatrix}
Z_c & X_c - i Y_c \\
X_c + i Y_c & -Z_c
\end{bmatrix}
$$

where `e_d` is the depolarizing amplitude, and $ X_c, Y_c, Z_c $ are the sampled components.

This matrix can be added to a unitary evolution or inserted into a stochastic noise model.



### Deterministic Relaxation Contribution

In addition to the stochastic contributions from amplitude damping, there is a deterministic component arising from the average (non-stochastic) part of the Lindblad evolution.

This is captured by the term:
$$
\Lambda := -\frac{\epsilon_1^2}{2} \int_0^1 \left( L_{\downarrow,s}^\dagger L_{\downarrow,s} \right) ds,
$$
which leads to a Hermitian drift term added to the noisy gate.

The corresponding integrals are:
$$
\begin{aligned}
\text{det}_1 &= \frac{\theta - \sin(\theta)}{2\theta}, \\
\text{det}_2 &= \frac{1 - \cos(\theta)}{\theta}, \\
\text{det}_3 &= \frac{\theta + \sin(\theta)}{2\theta},
\end{aligned}
$$

and they form the matrix:
$$
\texttt{deterministic} = -\frac{\epsilon_1^2}{2}
\begin{pmatrix}
\text{det}_1 & \frac{i}{2} e^{-i\phi} \text{det}_2 \\
-\frac{i}{2} e^{i\phi} \text{det}_2 & \text{det}_3
\end{pmatrix}
$$


In [39]:

# Define symbols
s, theta, phi = sp.symbols('s theta phi', real=True)
epsilon_1 = sp.symbols('epsilon_1', real=True)

# Your symbolic sigma_minus_s (2x2 matrix), assumed already computed
# sigma_minus_s = ...
# Hermitian conjugate
sigma_minus_s_dag = sigma_minus_s.H


# Compute L†L and simplify
product = sp.simplify(sigma_minus_s_dag * sigma_minus_s)
print("Inner product (L's) = ")
display(product)

# Integrate each element of the product matrix from s = 0 to 1
integrated = sp.Matrix(2, 2, lambda i, j: sp.simplify(sp.integrate(product[i, j], (s, 0, 1))))
display(integrated)

# Extract raw det1, det3 directly (diagonal)
det1 = sp.simplify(integrated[1, 1])  # lower-right
det3 = sp.simplify(integrated[0, 0])  # upper-left

# Compute the off-diagonal term
off_diag = sp.simplify(integrated[0, 1])

# Define the known complex prefactor explicitly
prefactor = -sp.I / 2 * sp.exp(-sp.I * phi)

# Isolate the real-valued part of det2 by factoring out the prefactor
# Multiply both sides symbolically instead of dividing
det2 = sp.simplify(sp.trigsimp(off_diag / prefactor))
det2 = sp.simplify(sp.trigsimp(det2.rewrite(sp.cos)))



# Construct Λ matrix in canonical Hermitian form
Λ = (-epsilon_1**2)/2 * sp.Matrix([
    [det1,  sp.I/2 * sp.exp(-sp.I * phi) * det2],
    [-sp.I/2 * sp.exp(sp.I * phi) * det2, det3]
])

# Display results
display(sp.Eq(sp.Symbol('det1'), det1))
display(sp.Eq(sp.Symbol('det2'), det2))
display(sp.Eq(sp.Symbol('det3'), det3))
display(sp.Symbol('Λ ='), Λ)


Inner product (L's) = 


Matrix([
[                                                                                                      cos(s*theta/2)**2, (I*(I*sin(phi) - cos(phi))*exp(2*I*phi)*cos(s*theta/2)**2 + (sin(phi) - I*cos(phi))*sin(s*theta/2)**2)*exp(-2*I*phi)*sin(s*theta/2)*cos(s*theta/2)],
[((sin(phi) + I*cos(phi))*exp(2*I*phi)*sin(s*theta/2)**2 + I*exp(I*phi)*cos(s*theta/2)**2)*sin(s*theta/2)*cos(s*theta/2),                                                                                                                                  sin(s*theta/2)**2]])

Matrix([
[                                         Piecewise(((theta + sin(theta))/(2*theta), (theta > 0) | (theta < 0)), (1, True)), Piecewise((I*(exp(2*I*theta) - 2*exp(I*theta) + 1)*exp(-I*(phi + theta))/(4*theta), Ne(theta**2*exp(2*I*phi), 0)), (0, True))],
[Piecewise((I*(-exp(2*I*theta) + 2*exp(I*theta) - 1)*exp(I*(phi - theta))/(4*theta), (theta > 0) | (theta < 0)), (0, True)),                                             Piecewise(((theta - sin(theta))/(2*theta), (theta > 0) | (theta < 0)), (0, True))]])

Eq(det1, Piecewise(((theta - sin(theta))/(2*theta), (theta > 0) | (theta < 0)), (0, True)))

Eq(det2, Piecewise(((1 - cos(theta))/theta, Ne(theta**2*exp(2*I*phi), 0)), (0, True)))

Eq(det3, Piecewise(((theta + sin(theta))/(2*theta), (theta > 0) | (theta < 0)), (1, True)))

Λ =

Matrix([
[       -epsilon_1**2*Piecewise(((theta - sin(theta))/(2*theta), (theta > 0) | (theta < 0)), (0, True))/2, -I*epsilon_1**2*Piecewise(((1 - cos(theta))/theta, Ne(theta**2*exp(2*I*phi), 0)), (0, True))*exp(-I*phi)/4],
[I*epsilon_1**2*Piecewise(((1 - cos(theta))/theta, Ne(theta**2*exp(2*I*phi), 0)), (0, True))*exp(I*phi)/4,          -epsilon_1**2*Piecewise(((theta + sin(theta))/(2*theta), (theta > 0) | (theta < 0)), (1, True))/2]])

### Variances and covariances for relaxation Itô processes depending on Z(t)
**Dephasing Noise — $ \sigma_z $ Channel**
Pure dephasing (phase damping) is modeled via a Lindblad operator:
$$
L_\phi(s) = \sqrt{\frac{1}{T_2}} \, \sigma_z(s),
$$
where $ \sigma_z(s) $ is the Pauli-Z operator in the interaction picture.

As with other stochastic terms, we expand this in second-order Itô integrals. The statistical quantities used to build the covariance matrix are:
$$
\begin{aligned}
\mathbb{V}[\mathcal{I}_1] &= \frac{2\theta + \sin(2\theta)}{4\theta}, \\
\mathbb{V}[\mathcal{I}_2] &= \frac{2\theta - \sin(2\theta)}{4\theta}, \\
\text{Cov}[\mathcal{I}_1, \mathcal{I}_2] &= \frac{\sin^2(\theta)}{2\theta}.
\end{aligned}
$$

This results in a 2×2 Hermitian contribution matrix $ \texttt{Ip} $, scaled by the dephasing amplitude:
$$
\epsilon_\phi = \sqrt{\frac{1}{2} \left( \frac{t_g}{T_2} - \frac{t_g}{2T_1} \right)}.
$$

In [40]:
Z_decomp = pauli_decomposition(Z_s, paulis)

# Compute variances/covariance
def var_and_cov(decomp):
    fz = decomp['Z']
    fy = decomp['Y']
    var_z = sp.integrate(fz**2, (s, 0, 1))
    var_y = sp.integrate(fy**2, (s, 0, 1))
    cov_zy = sp.integrate(fz * fy, (s, 0, 1))
    return fz, fy, sp.simplify(var_z), sp.simplify(var_y), sp.simplify(cov_zy)

fZz, fZy, var_Z, var_Y, cov_ZY = var_and_cov(Z_decomp)

fZz = sp.simplify(fZz)
fZy = sp.simplify(fZy)

var_Z = sp.trigsimp(var_Z, trig=True)
var_Z = var_Z.rewrite(sp.sin(2*theta))  # Force rewrite if needed
var_Z = sp.simplify(var_Z)

Var_Z2_sin2theta = sp.trigsimp(var_Y, trig=True)
Var_Z2_sin2theta = Var_Z2_sin2theta.rewrite(sp.sin(2*theta))  # Force rewrite if needed
Var_Z2_sin2theta = sp.simplify(Var_Z2_sin2theta)

# Display results
display(sp.Eq(sp.Symbol("Var_Z1"), var_Z))
display(sp.Eq(sp.Symbol("Var_Z2"), Var_Z2_sin2theta))
display(sp.Eq(sp.Symbol("Cov_Z12"), cov_ZY))

display(sp.Eq(sp.Symbol("f_Zz"), fZz))
display(sp.Eq(sp.Symbol("f_Zy"), fZy))

Eq(Var_Z1, Piecewise(((2*theta + sin(2*theta))/(4*theta), (theta > 0) | (theta < 0)), (1, True)))

Eq(Var_Z2, Piecewise(((2*theta - sin(2*theta))*cos(phi)**2/(4*theta), (theta > 0) | (theta < 0)), (0, True)))

Eq(Cov_Z12, Piecewise((sin(theta)**2*cos(phi)/(2*theta), (theta > 0) | (theta < 0)), (0, True)))

Eq(f_Zz, cos(s*theta))

Eq(f_Zy, sin(s*theta)*cos(phi))

---
